In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json

/Users/dummy/Downloads/Langchain Projects/1-QnA Chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
openai = OpenAI()

In [7]:
system_message = 'You are a helpful AI assistant'
system_message += 'Give short and courteous single line answers'
system_message += 'Always be accurate. If you dont know, say that you dont know'
MODEL = 'gpt-4o-mini'

def chat(message,history):
    messages =[{'role':'system','content':system_message}]
    for human,assistant in history:
        messages.append({'role':'user','content':human})
        messages.append({'role':'assistant','content':assistant})
    messages.append({'role':'user','content':message})
    response = openai.chat.completions.create(model = MODEL,messages = messages)    
    return response.choices[0].message.content


gr.ChatInterface(fn=chat).launch()


/Users/dummy/Downloads/Langchain Projects/1-QnA Chatbot/venv/lib/python3.10/site-packages/gradio/components/chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


In [16]:
ticket_prices ={"london":"$799","bangalore":"$1099","berlin":"$899"}

def get_ticket_price(destination):
    print(f'Tool get_ticket_price called for {destination}')
    city = destination.lower()
    return ticket_prices.get(city,'unknown')


get_ticket_price('bangalore')

Tool get_ticket_price called for bangalore


'$1099'

In [17]:
get_ticket_price('berlin')

Tool get_ticket_price called for berlin


'$899'

In [18]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [19]:
tools = [{"type": "function", "function": price_function}]

In [20]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [21]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for NY
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Paris
Tool get_ticket_price called for London
